# Q4 audit: rebuild P&L

In [6]:
import sys  # no installation needed for stdlib/project-local
from pathlib import Path  # no installation needed for stdlib/project-local

ROOT = Path(r"C:\\Users\\quantbase\\Desktop\\ecom_forecast")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


In [7]:
import pandas as pd  # already in env ? no new install for third-party libs

from src.config import ProjectPaths  # no installation needed for stdlib/project-local
from src.audit import (  # no installation needed for stdlib/project-local
    build_q4_daily_pnl,
    compute_weighted_cogs_pct,
    summarize_q4,
)


In [8]:
paths = ProjectPaths.from_root()
paths.ensure_directories()
audit_dir = paths.outputs_dir / 'audit'
audit_dir.mkdir(parents=True, exist_ok=True)
assumptions = paths.load_assumptions()
acct = assumptions.get("accounting", {})
shipping_per_order = float(acct.get("shipping_per_order", 0.0))
if shipping_per_order <= 0:
    raise ValueError("shipping_per_order must be > 0; check assumptions.yaml (accounting.shipping_per_order)")




In [9]:
sales_daily = pd.read_pickle(paths.clean_dir / 'sales_daily.pkl')
sessions_daily = pd.read_pickle(paths.clean_dir / 'sessions_daily.pkl')
marketing_daily = pd.read_pickle(paths.clean_dir / 'marketing_daily.pkl')
billing_location = pd.read_pickle(paths.clean_dir / 'billing_location.pkl')
variant = pd.read_pickle(paths.clean_dir / 'variant.pkl')

for frame in (sales_daily, sessions_daily, marketing_daily):
    if 'Day' in frame.columns:
        frame['Day'] = pd.to_datetime(frame['Day'])


In [10]:
total_orders = pd.to_numeric(billing_location['Orders'], errors='coerce').fillna(0).sum()
weighted = compute_weighted_cogs_pct(variant)

q4_daily = build_q4_daily_pnl(
    sales_df=sales_daily,
    sessions_df=sessions_daily,
    marketing_df=marketing_daily,
    total_orders=total_orders,
    weighted_cogs_pct=weighted['weighted_cogs_pct'],
    shipping_per_order=shipping_per_order,
)

assert len(q4_daily) == len(sales_daily)
assert abs(q4_daily['Orders_est'].sum() - total_orders) <= 1e-6

q4_summary = summarize_q4(
    q4_daily,
    variant_cogs_total=weighted['variant_cogs_total'],
    variant_net_sales_total=weighted['variant_net_sales_total'],
)
q4_summary

,NetSales_total,Orders_total_est,Shipping_total,AdSpend_total,WeightedCOGS_pct,COGS_est_total,CM_total,CM_pct,variant_cogs_total,cogs_est_minus_variant_cogs,variant_net_sales_total
0,2705295.6,15093.0,128290.5,627400.0,0.415243,1.123356e+06,826248.919755,0.305419,1126242.94,-2886.759755,2712247.57


In [11]:
cash_burn_day = q4_daily.loc[[q4_daily['CM$'].idxmin()]].copy()
avg_rps = q4_daily['RPS'].mean()
efficiency_leak_days = (
    q4_daily.assign(RPS_vs_avg=q4_daily['RPS'] - avg_rps)
    .sort_values('Ad_Spend', ascending=False)
    .head(10)
)
cash_burn_day


,Day,Net sales,Gross sales,Returns,Discounts,Sessions,Conversion rate,Ad_Spend,Meta_Spend,Google_Spend,TikTok_Spend,Email_SMS_Cost,Orders_est,Shipping_cost,COGS_est,CM$,CM%,RPS
6,2025-09-23,17362.43,39063.49,-21516.07,-184.99,7558,0.015348,15050,9500,3200,2100,250,163.701543,1391.463114,7209.634705,-6288.667818,-0.3622,2.297225


In [12]:
efficiency_leak_days.head(5)

,Day,Net sales,Gross sales,Returns,Discounts,Sessions,Conversion rate,Ad_Spend,Meta_Spend,Google_Spend,TikTok_Spend,Email_SMS_Cost,Orders_est,Shipping_cost,COGS_est,CM$,CM%,RPS,RPS_vs_avg
72,2025-11-28,43808.32,48447.99,-2975.77,-1663.90,7331,0.025099,15500,9000,4000,2000,500,259.664516,2207.148387,18191.116349,7910.055264,0.180561,5.975763,1.458232
6,2025-09-23,17362.43,39063.49,-21516.07,-184.99,7558,0.015348,15050,9500,3200,2100,250,163.701543,1391.463114,7209.634705,-6288.667818,-0.362200,2.297225,-2.220306
16,2025-10-03,23256.61,28551.90,-5295.29,0.00,10849,0.008757,12800,5500,2000,5000,300,134.065919,1139.560309,9657.154129,-340.104438,-0.014624,2.143664,-2.373868
71,2025-11-27,58084.78,63104.66,-3651.18,-1368.70,7002,0.037418,12400,7500,3000,1500,400,369.739691,3142.787377,24119.322336,18422.670287,0.317169,8.295456,3.777924
48,2025-11-04,67291.00,81634.52,-14015.72,-327.80,9291,0.040469,10500,6500,2500,1200,300,530.618794,4510.259748,27942.144557,24338.595695,0.361692,7.242600,2.725069


In [13]:
q4_daily.to_csv(audit_dir / 'q4_daily.csv', index=False)
q4_summary.to_csv(audit_dir / 'q4_summary.csv', index=False)
cash_burn_day.to_csv(audit_dir / 'cash_burn_day.csv', index=False)
efficiency_leak_days.to_csv(audit_dir / 'efficiency_leak_days.csv', index=False)
['q4_daily.csv', 'q4_summary.csv', 'cash_burn_day.csv', 'efficiency_leak_days.csv']


['q4_daily.csv',
 'q4_summary.csv',
 'cash_burn_day.csv',
 'efficiency_leak_days.csv']